# Visualizing Spins in 3D with Julia and GLMakie

***Using Julia 1.9.3 version on macOS Sonoma 14.0, Apple M1 Pro chip.***

Install the necessary packages if not already done:

In [12]:
# using Pkg
# Pkg.add("GLMakie")
# Pkg.add("GeometryBasics")
# Pkg.add("Colors")

Import them at the start of the notebook:

In [2]:
using GLMakie, GeometryBasics, Colors

GLMakie needs to be activated to render the graphics.

In [13]:
GLMakie.activate!()

### The Parameters
Create a struct for parameters:

In [14]:
struct Par
    N::Int64      # number of cells across the lattice
    L::Float64    # length of the elements
    J::Float64   # NN distance (=1 here)
    β::Float64    # angle spins make with the x axis 
end

C = Par(3, 1.0, 1.0, π/4)

Par(3, 1.0, 1.0, 0.7853981633974483)

### Functions

Generate lattice points (position vectors for each spin to get assigned to):

In [15]:
function get_spin_centers(C::Par)

    # generate a periodic lattice of points

    # side length l in terms of J spacing
    # l is the distance from outermost spin centers on each side
    
    l = (2C.N - 1) * C.J

    spin_centers = Vector{Float64}[]
    
    for x = -l/2 : C.J : l/2 
        for y = l/2 : -C.J : -l/2
            push!(spin_centers, Float32[x,y,0.0])
        end
    end

    return spin_centers
end

get_spin_centers (generic function with 1 method)

In [16]:
spin_centers = get_spin_centers(C)

36-element Vector{Vector{Float64}}:
 [-2.5, 2.5, 0.0]
 [-2.5, 1.5, 0.0]
 [-2.5, 0.5, 0.0]
 [-2.5, -0.5, 0.0]
 [-2.5, -1.5, 0.0]
 [-2.5, -2.5, 0.0]
 [-1.5, 2.5, 0.0]
 [-1.5, 1.5, 0.0]
 [-1.5, 0.5, 0.0]
 [-1.5, -0.5, 0.0]
 ⋮
 [1.5, -0.5, 0.0]
 [1.5, -1.5, 0.0]
 [1.5, -2.5, 0.0]
 [2.5, 2.5, 0.0]
 [2.5, 1.5, 0.0]
 [2.5, 0.5, 0.0]
 [2.5, -0.5, 0.0]
 [2.5, -1.5, 0.0]
 [2.5, -2.5, 0.0]

Generate an **ordered** spin configuration.

In [7]:
function create_T2_config(C::Par)

    # individual spin vectors grouped in "cells" of 4
    spin_1 = [cos(C.β), -sin(C.β), 0]
    spin_2 = [cos(C.β), cos(C.β), 0]
    spin_3, spin_4 = spin_2, spin_1

    # number of columns in the lattice (it is also a # of spins per side)
    columns = 2C.N

    μ_hat = Vector{Float64}[]

    for col = 1:columns
        spins = isodd(col) ? [spin_1, spin_2] : [spin_3, spin_4]
        append!(μ_hat, repeat(spins, C.N))
    end

    return μ_hat
end

create_T2_config (generic function with 1 method)

In [8]:
μ_hat = create_T2_config(C)

36-element Vector{Vector{Float64}}:
 [0.7071067811865476, -0.7071067811865475, 0.0]
 [0.7071067811865476, 0.7071067811865476, 0.0]
 [0.7071067811865476, -0.7071067811865475, 0.0]
 [0.7071067811865476, 0.7071067811865476, 0.0]
 [0.7071067811865476, -0.7071067811865475, 0.0]
 [0.7071067811865476, 0.7071067811865476, 0.0]
 [0.7071067811865476, 0.7071067811865476, 0.0]
 [0.7071067811865476, -0.7071067811865475, 0.0]
 [0.7071067811865476, 0.7071067811865476, 0.0]
 [0.7071067811865476, -0.7071067811865475, 0.0]
 ⋮
 [0.7071067811865476, 0.7071067811865476, 0.0]
 [0.7071067811865476, -0.7071067811865475, 0.0]
 [0.7071067811865476, 0.7071067811865476, 0.0]
 [0.7071067811865476, 0.7071067811865476, 0.0]
 [0.7071067811865476, -0.7071067811865475, 0.0]
 [0.7071067811865476, 0.7071067811865476, 0.0]
 [0.7071067811865476, -0.7071067811865475, 0.0]
 [0.7071067811865476, 0.7071067811865476, 0.0]
 [0.7071067811865476, -0.7071067811865475, 0.0]

Generate a **random** spin configuration.

In [9]:
function create_random_config(C::Par)

    # individual spin vectors grouped in "cells" of 4
    spin_1 = [cos(C.β), -sin(C.β), 0]
    spin_2 = [cos(C.β), cos(C.β), 0]
    spin_3, spin_4 = spin_2, spin_1

    # number of columns in the lattice (it is also a # of spins per side)
    columns = 2C.N
    
    μ_hat_rand = Vector{Float64}[]

    for col = 1:columns
        spins = isodd(col) ? [spin_1 *rand([1,-1]) , spin_2 *rand([1,-1])] : [spin_3 *rand([1,-1]), spin_4 *rand([1,-1])]
        append!(μ_hat_rand, repeat(spins, C.N))
    end

    return μ_hat_rand
end

create_random_config (generic function with 1 method)

In [10]:
μ_hat_rand = create_random_config(C)

36-element Vector{Vector{Float64}}:
 [0.7071067811865476, -0.7071067811865475, 0.0]
 [-0.7071067811865476, -0.7071067811865476, -0.0]
 [0.7071067811865476, -0.7071067811865475, 0.0]
 [-0.7071067811865476, -0.7071067811865476, -0.0]
 [0.7071067811865476, -0.7071067811865475, 0.0]
 [-0.7071067811865476, -0.7071067811865476, -0.0]
 [-0.7071067811865476, -0.7071067811865476, -0.0]
 [0.7071067811865476, -0.7071067811865475, 0.0]
 [-0.7071067811865476, -0.7071067811865476, -0.0]
 [0.7071067811865476, -0.7071067811865475, 0.0]
 ⋮
 [-0.7071067811865476, -0.7071067811865476, -0.0]
 [-0.7071067811865476, 0.7071067811865475, -0.0]
 [-0.7071067811865476, -0.7071067811865476, -0.0]
 [-0.7071067811865476, -0.7071067811865476, -0.0]
 [-0.7071067811865476, 0.7071067811865475, -0.0]
 [-0.7071067811865476, -0.7071067811865476, -0.0]
 [-0.7071067811865476, 0.7071067811865475, -0.0]
 [-0.7071067811865476, -0.7071067811865476, -0.0]
 [-0.7071067811865476, 0.7071067811865475, -0.0]

Now we have the data to plot.

In [11]:
function plot_spins(C::Par)

    spin_centers = get_spin_centers(C)
    μ_hat = create_T2_config(C)
    #μ_hat_rand = create_random_config(C)

    spins_per_side = 2C.N
    num_spins = spins_per_side^2

    spin_center = [Point3f(i) for i in spin_centers] 
    spin_conf = [(C.L/2) * Vec3f(i) for i in μ_hat]
    #spin_conf = [(C.L/2) * Vec3f(i) for i in μ_hat_rand]

    fig = Figure(resolution=(720, 720), dpi=600)
    ax = Axis3(fig[1,1], aspect=:data, perspectiveness=0.7)

    arrows!(ax, spin_center, spin_conf, 
            linecolor=:darkgrey, 
            arrowcolor=RGB(0.43, 0.43, 0.43), 
            quality=32, 
            arrowsize=Vec3f0(0.3, 0.3, C.L/2), 
            linewidth=0.15, 
            align=:center,)

    hidedecorations!(ax)
    hidespines!(ax)

    save("$(num_spins)_orderedConfig_spins3D.png", fig)
    # save("$(num_spins)_randConfig_spins3D.png", fig)

    return fig
end

plot_spins(C)
